In [ ]:
import gc
import matrix_methods as mm
import torch

from models import GAN, SL2Z_DataModule
from os import cpu_count
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
def train(config = None):
	gc.collect()
	torch.cuda.empty_cache()

	model = GAN(config=config)
	datamodule = SL2Z_DataModule(config=config)
	logger = TensorBoardLogger(save_dir='./sl2z-gan-logs')
	trainer = Trainer(
		#fast_dev_run = True,
		accelerator = 'gpu',
		devices = 'auto',
		max_epochs = config['epochs'],
		logger = logger,
		benchmark = True,
		enable_progress_bar = False # workaround for Jupyter websocket timeout bug
	)
	trainer.fit(model, datamodule)
	return trainer, model, datamodule

In [ ]:
train_config = {
	'epochs': 900,
	'lr': 1e-3,
    'noise_dim': 8,
    'noise_sigma': 1e5,
    'packing_degree': 16,
	'data_path': 'data.npy',
	'batch_size': 8192,
	'num_workers': cpu_count()
}

In [ ]:
trainer, model, datamodule = train(train_config)

In [ ]:
samples = model.generate_samples(20)

In [ ]:
torch.round(samples)

In [ ]:
mm.det_multi(torch.round(samples).detach().numpy())